In [309]:
import pymongo
from dotenv import load_dotenv
import os

load_dotenv()
mongodb_uri = os.getenv("MONGODB_URI")
client = pymongo.MongoClient(mongodb_uri)
db = client[os.environ.get("MONGODB_DB")]
classes = db["classes"]
locations = db["locations"]

In [310]:
ece_120 = classes.find_one({"code" : "ECE", "number" : "120"})
ece_110 = classes.find_one({"code" : "ECE", "number" : "110"})
phys_211 = classes.find_one({"code" : "PHYS", "number" : "211"})

ece_120_crns = [section["crn"] for section in ece_120["sections"]]
ece_110_crns = [section["crn"] for section in ece_110["sections"]]
phys_211_crns = [section["crn"] for section in phys_211["sections"]]

In [329]:
from datetime import datetime, timedelta

classes_to_take = [
    {
        "code" : "ECE",
        "number" : "110",
        "crn_list" : ece_110_crns
    },
    {
        "code" : "ECE",
        "number" : "120",
        "crn_list" : ece_120_crns
    },
    {
        "code" : "PHYS",
        "number" : "211",
        "crn_list" : phys_211_crns
    }
]



In [330]:
print(classes_to_take)

[{'code': 'ECE', 'number': '110', 'crn_list': ['36785', '36788', '36780', '36801', '36794', '55569', '36798', '55155', '36781', '55156', '36778', '36800', '36792', '36783', '36796', '62483', '55568', '36790', '36789', '62844', '59864', '59865', '59866', '59867', '59868', '59869', '59870', '59871', '59872', '59873', '59875', '59876', '59878', '59879', '59880', '62509']}, {'code': 'ECE', 'number': '120', 'crn_list': ['64596', '64597', '64598', '64599', '65253', '65254', '65255', '65256', '65257', '65258', '65260', '65261', '66763', '64595', '65733', '65734', '75613']}, {'code': 'PHYS', 'number': '211', 'crn_list': ['55650', '34564', '34566', '59136', '59140', '34590', '34569', '34598', '54863', '34592', '34624', '47717', '34595', '60539', '34601', '54864', '60540', '34608', '34611', '60541', '47577', '34630', '34709', '34604', '34613', '34616', '34619', '34586', '34621', '59137', '55008', '34625', '34727', '52599', '56971', '60543', '60544', '55801', '60546', '60545', '56990', '55007', '

In [312]:
# User Prefs
start_time = 8 #8am
end_time = 17 #5pm
open_sections_only = False
max_distance = 500 #meters
back_to_back = True
pref_time = 12 #noon
pref_sections = ['36785', '36788', '36780', '36801']
lunch = {
    "start" : 11,
    "end" : 14,
    "duration" : 1
}

In [313]:
for clas in classes_to_take : 
    clas_mongo_object = classes.find_one({"code" : clas["code"], "number" : clas["number"]})
    clas["sections"] = []
    for section in clas_mongo_object["sections"] :
        if section["crn"] in clas["crn_list"] :
            section["class"] = {
                "code" : clas["code"],
                "number" : clas["number"]
            }
            clas["sections"].append(section)

print(classes_to_take)

[{'code': 'ECE', 'number': '110', 'crn_list': ['36785', '36788', '36780', '36801', '36794', '55569', '36798', '55155', '36781', '55156', '36778', '36800', '36792', '36783', '36796', '62483', '55568', '36790', '36789', '62844', '59864', '59865', '59866', '59867', '59868', '59869', '59870', '59871', '59872', '59873', '59875', '59876', '59878', '59879', '59880', '62509'], 'sections': [{'crn': '36785', 'api_link': 'https://courses.illinois.edu/cisapp/explorer/schedule/2023/fall/ECE/110/36785.xml', 'section_number': 'AB0', 'status_code': 'A', 'part_of_term': '1', 'section_status_code': 'A', 'enrollment_status': 'Closed', 'section_text': 'ECE 110 cannot be taken concurrently with ECE 120.', 'start_date': datetime.datetime(2023, 8, 21, 0, 0), 'end_date': datetime.datetime(2023, 12, 6, 0, 0), 'meetings': [{'id': '0', 'type': 'Laboratory', 'type_code': 'LAB', 'start_time': datetime.datetime(1900, 1, 1, 18, 0), 'end_time': datetime.datetime(1900, 1, 1, 20, 50), 'days': 'M', 'room_number': '1001'

In [314]:
def check_for_no_section (classes_to_take) :
    for clas in classes_to_take :
        print(f" {clas['code']} {clas['number']} has {len(clas['sections'])} sections")
        if len(clas["sections"]) == 0 :
            raise Exception("No sections found for " + clas["code"] + " " + clas["number"])

## Hard Filters 
Straight Up Delete CRNs

In [315]:
# Remove sections that are not in preferred times

def is_preferred_time (section) :
    earliest_meeting = section["meetings"][0]['start_time']
    latest_meeting = section["meetings"][0]['end_time']
    for meeting in section["meetings"] :
        if meeting['start_time'] < earliest_meeting :
            earliest_meeting = meeting['start_time']
        if meeting['end_time'] > latest_meeting :
            latest_meeting = meeting['end_time']

    return earliest_meeting.hour > start_time and latest_meeting.hour < end_time


for clas in classes_to_take :
    clas["sections"] = [section for section in clas['sections'] if is_preferred_time(section)]

check_for_no_section(classes_to_take)

 ECE 110 has 24 sections
 ECE 120 has 16 sections
 PHYS 211 has 42 sections


In [316]:
# Removes sections that are not open if open_sections_only is True

if open_sections_only :
    for clas in classes_to_take :
        clas["sections"] = [section for section in clas['sections'] if section['enrollment_status'].lower() != 'closed']

    check_for_no_section(classes_to_take)

## Generate All Possible Schedules

In [317]:
groups = {}

for clas in classes_to_take :
    base_group_name = f"{clas['code']}_{clas['number']}_"
    for section in clas['sections'] :
        list_of_meeting_types = [meeting['type_code'] for meeting in section['meetings']]
        group_key = base_group_name + "_".join([str(x) for x in list_of_meeting_types])
        groups[group_key] = groups.get(group_key, []) + [section]

print(groups)

{'ECE_110_LAB': [{'crn': '36788', 'api_link': 'https://courses.illinois.edu/cisapp/explorer/schedule/2023/fall/ECE/110/36788.xml', 'section_number': 'AB1', 'status_code': 'A', 'part_of_term': '1', 'section_status_code': 'A', 'enrollment_status': 'CrossListOpen (Restricted)', 'section_text': 'ECE 110 cannot be taken concurrently with ECE 120.', 'start_date': datetime.datetime(2023, 8, 21, 0, 0), 'end_date': datetime.datetime(2023, 12, 6, 0, 0), 'meetings': [{'id': '0', 'type': 'Laboratory', 'type_code': 'LAB', 'start_time': datetime.datetime(1900, 1, 1, 9, 0), 'end_time': datetime.datetime(1900, 1, 1, 11, 50), 'days': 'W', 'room_number': '1001', 'building_name': 'Electrical & Computer Eng Bldg', 'coordinates': [40.115077164725506, -88.22815195075593]}], 'class': {'code': 'ECE', 'number': '110'}}, {'crn': '36780', 'api_link': 'https://courses.illinois.edu/cisapp/explorer/schedule/2023/fall/ECE/110/36780.xml', 'section_number': 'AB2', 'status_code': 'A', 'part_of_term': '1', 'section_stat

In [318]:
import itertools

possible_schedules = list(itertools.product(*groups.values()))
print(f"{len(possible_schedules)} schedules")

2332800 schedules


## Rank Schedules

In [319]:
def convert_to_time_based (list_of_sections) : 
    schedule_slots = {
        'W': [None]*23,
        'R': [None]*23,
        'F': [None]*23,
        'T': [None]*23,
        'M': [None]*23,
    }

    for section in list_of_sections :
        for meeting in section['meetings'] :
            start_hour = meeting['start_time'].hour
            end_hour = meeting['end_time'].hour
            for day in list(meeting['days'].strip()) :
                for hour in range(start_hour, end_hour+1) :
                    section_copy = section  
                    section_copy['meetings'] = [meeting]
                    if schedule_slots[day][hour] is not None :
                        return None
                    schedule_slots[day][hour] = section_copy

    return schedule_slots

In [320]:
import haversine as hs

distance_weight = 25

def list_of_successive_distances (schedule) :

    list_of_distances = []
    for day in schedule :
        for hour in range(0, 23) :
            if schedule[day][hour] is not None and schedule[day][hour + 1] is not None :
                coords1 = schedule[day][hour]['meetings'][0]['coordinates']
                coords2 = schedule[day][hour + 1]['meetings'][0]['coordinates']
                if coords1 is not None and coords2 is not None :
                    list_of_distances.append(hs.haversine(coords1, coords2, unit=hs.Unit.METERS))

    return [x for x in list_of_distances if x != 0.0]

def distance_score (list_of_sections) -> float :
    # Returns a score between -distance_weight and distance_weight based on how far apart the classes are
    list_of_distances =  list_of_successive_distances(list_of_sections)

    if len(list_of_distances) == 0 :
        return 0
    
    mean_distance = sum(list_of_distances) / len(list_of_distances)
    
    # Find the distance score
    if mean_distance > max_distance :
        percent_over = (mean_distance - max_distance) / max_distance
        return -(percent_over * distance_weight)
    elif mean_distance < max_distance :
        percent_under = (max_distance - mean_distance) / max_distance
        return percent_under * distance_weight
    else :
        return 0

In [321]:
back_to_back_weight = 15

def back_to_back_score (list_of_sections) -> float :
    back_to_back_count = 0
    total_count = 0

    for day in list_of_sections :
        for hour in range(0, 23) :
            if list_of_sections[day][hour] is not None and list_of_sections[day][hour + 1] is not None :
                total_count += 1
                if list_of_sections[day][hour]['crn'] != list_of_sections[day][hour + 1]['crn'] :
                    back_to_back_count += 1

    if total_count == 0 :
        return 0
    elif back_to_back :
        return (back_to_back_count / total_count) * back_to_back_weight
    else :
        return (-back_to_back_count / total_count) * back_to_back_weight

In [322]:
time_weight = 50

def time_score(list_of_sections) :
    time_list = []

    for day in list_of_sections :
        for hour in range(0, 23) :
            if list_of_sections[day][hour] is not None :
                time_list.append(hour)

    if len(time_list) == 0 :
        return 0
    else :
        mean_time = sum(time_list) / len(time_list)
        return (1 - (abs(mean_time - pref_time) / pref_time)) * time_weight
    

In [323]:
section_weight = 10

def section_score (list_of_sections) :
    section_count = 0
    for day in list_of_sections :
        for hour in range(0, 23) :
            if list_of_sections[day][hour] is not None and list_of_sections[day][hour]['crn'] in pref_sections:
                section_count += 1

    return section_count * section_weight

In [324]:
lunch_weight = 7.5

def lunch_score (list_of_sections) :
    lunch_count = 0
    for day in list_of_sections :
        for hour in range(0, 23) :
            if list_of_sections[day][hour] is None and hour >= lunch['start'] and hour < lunch['end'] :
                duration_hours = [list_of_sections[day][x] is None for x in range(hour, hour + lunch['duration']) if x < 23]
                if all(duration_hours) :
                    lunch_count += 1
                    break
    return (lunch_count - 5) * lunch_weight

In [325]:
import json
def rank (list_of_sections) :
    schedule = convert_to_time_based(list_of_sections)
    if schedule is None :
        return None
    ds = distance_score(schedule)
    btb = back_to_back_score(schedule)
    ts = time_score(schedule)
    ss = section_score(schedule)
    ls = lunch_score(schedule)
    return ds + btb + ts + ss + ls

In [326]:
sorted_schedules = []
for schedule in possible_schedules :
    score = rank(schedule)
    if score is not None :
        sorted_schedules.append(
            {
                "schedule" : schedule,
                "score" : score
            }
        )

sorted_schedules.sort(key=lambda x: x["score"], reverse=True)

In [327]:
with open('schedule.json', 'w') as f:
    json.dump(sorted_schedules[:5], f, indent=4, default=str)